In [2]:
# Standard libraries
import h5py
import numpy as np
import matplotlib.pyplot as plt
from bisect import bisect_left

# Append base directory
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path1p = os.path.dirname(currentdir)
path2p = os.path.dirname(path1p)
libpath = os.path.join(path1p, "lib")
pwd_mat = os.path.join(os.path.join(path2p, "data/"), "sim_ds_mat")
pwd_h5 = os.path.join(os.path.join(path2p, "data/"), "sim_ds_h5")

sys.path.insert(0, libpath) 
print("Appended library directory", libpath)

# User libraries
from matlab.matlab_yaro_lib import read_mat
from signal_lib import downsample
from corr_lib import sprMat
from idtxl_wrapper import idtxlParallelCPU, idtxlResultsParse
from qt_wrapper import gui_fpath


# IDTxl libraries
from idtxl.bivariate_mi import BivariateMI
from idtxl.multivariate_mi import MultivariateMI
from idtxl.bivariate_te import BivariateTE
from idtxl.multivariate_te import MultivariateTE
from idtxl.data import Data
from idtxl.visualise_graph import plot_network

/opt/anaconda3/envs/py36qt5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Appended library directory /media/aleksejs/DataHDD/work/codes/theoretical-models/experimental-bridge/nest-mesoscopic-metric-test/codes/lib


## 1. Load Neuronal Data

The neuronal data for a given experiment is given by the 3D matrix of dimensions `[N_Trial, N_Times, N_Channels]`

Notes:
* `DT = 0.05` : the sampling interval is always 50ms
* `N_Times = 201` : the total experiment time is always 10s. All experiments are aligned

In [5]:
# Declare constants
params = {
    "exp_timestep" : 0.05, # 50ms, the standard measurement interval
    "bin_timestep" : 0.2,  # 200ms, the binned interval
    
    # The standard timesteps for different scenarios
    "range_CUE" : (1.0, 1.5),    # 1-1.5 seconds trial time
    "range_TEX" : (2.0, 3.5),    # 2-3.5 seconds trial time
    "range_LIK" : (3.5, 6.0),    # 3.5-6 seconds trial time
    "range_ALL" : (0.0, 10.0),   # 0-10  seconds

#     "samples_window" : "ALL",
#     "trials_type"    : "GO",
    "subsampling"    : True,   # To bin or not to bin data
#     "subsampling_windows_lenghts" : 4*np.ones((1,51)),  #uniform subsampling, 50 bins
    
    "te_delay_min" : 1,
    "te_delay_max" : 5,
    
#     "te_delay_stepsize" : 1,
#     "te_receiver_embedding_tau" : 1,
#     "te_sampling_rate" : 20
}

# GUI: Select videos for training
pwd_tmp = "./"
next_path = "AAAAA"
params['pwd_data_folders'] = []
while next_path != '':
    print("Currently using data folders:", params['pwd_data_folders'])
    next_path = gui_fpath("Select one more data directory, or cancel", pwd_tmp)
    if next_path != '':
        params['pwd_data_folders'] += [next_path]
        pwd_tmp = os.path.dirname(next_path)  # Next time choose from parent folder
print('Done')

Currently using data folders: []
Currently using data folders: ['/home/aleksejs/Documents']
Currently using data folders: ['/home/aleksejs/Documents', '/home/aleksejs/Pictures']
Done


In [3]:
for folderPathName in params['pwd_data_folders']:
    folderName = os.path.basename(folderPathName)
    #############################
    # Reading and parsing
    #############################

    # Read LVM file from command line
    data, behaviour = read_mat(folderPathName)

    # Get parameters
    nTrials, nTimes, nChannels = data.shape
    print("Loaded neuronal data with (nTrials, nTimes, nChannels)=", data.shape)

    assert nTimes == 201, "The number of timesteps must be 201 for all data, got "+str(nTimes)
    nTimesDownsampled = int(nTimes * params["exp_timestep"] / params["bin_timestep"])
    print(nTimesDownsampled)

    # Timeline (x-axis)
    tlst = params["exp_timestep"] * np.linspace(0, nTimes, nTimes)

    # Downsample data
    print("Downsampling from", params["exp_timestep"], "ms to", params["bin_timestep"], "ms")
    if params["subsampling"]:
        params["timestep"] = params["bin_timestep"]
        tlst_new = downsample(tlst, data[0, :, 0], nTimesDownsampled)[0]
        data_new = np.array([[downsample(tlst, data[i, :, j], nTimesDownsampled)[1] 
                              for i in range(nTrials)] 
                             for j in range(nChannels)])
    else:
        params["timestep"] = params["exp_timestep"]

    # Replace old data with subsampled one    
    tlst, data = tlst_new, data_new.transpose(1, 2, 0)
    nTrials, nTimes, nChannels = data.shape
    print("After downsampling data shape is (nTrials, nTimes, nChannels)=", data.shape)
    
    #############################
    # Analysis
    #############################
    
    methods_lst = ["BivariateTE", "MultivariateTE"]
    dataRanges = ["range_CUE", "range_TEX", "range_LIK"]
    
    idtxl_settings = {
        'dim_order'       : 'rsp',
        'cmi_estimator'   : 'JidtGaussianCMI',
        'max_lag_sources' : params["te_delay_max"],
        'min_lag_sources' : params["te_delay_min"]
   }
    
    for dataRange in dataRanges:
        idx_left  = bisect_left(tlst, params[dataRange][0])  # Inclusive
        idx_right = bisect_left(tlst, params[dataRange][1])  # Non-inclusive (exactly what we want)
        print("Range for CUE", params[dataRange], "approximated to", (tlst[idx_left], tlst[idx_right-1]))
        dataThis = data[:, idx_left:idx_right, :]
        
        # Correct for max-lag, if the slice is too small
        idtxl_settings['max_lag_sources'] = min([idtxl_settings['max_lag_sources'], dataThis.shape[1]-1])
    
        for method in methods_lst:
            idtxl_settings['method'] = method
            
            rez = idtxlParallelCPU(dataThis, idtxl_settings)
    
            TE, lag, p = idtxlResultsParse(rez, nChannels, method=idtxl_settings['method'], storage='matrix')
        
            #######################
            # Save results to file
            #######################
            savename = os.path.join(pwd_h5, folderName + '_' + method + '_' + dataRange + '.h5')

            h5f = h5py.File(savename, "w")

            grp_rez = h5f.create_group("results")
            grp_rez['TE_table'] = TE
            grp_rez['delay_table'] = lag
            grp_rez['p_table'] = p

            h5f.close()

Reading Yaro data from /mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15/mtp_15_2018_05_16_a
Loaded neuronal data with (nTrials, nTimes, nChannels)= (338, 201, 12)
50
Downsampling from 0.05 ms to 0.2 ms
After downsampling data shape is (nTrials, nTimes, nChannels)= (338, 50, 12)
Range for CUE (1.0, 1.5) approximated to (1.0859025, 1.4899125)
Adding data with properties: 12 processes, 3 samples, 338 replications
overwriting existing data

Target: 4 - testing sources [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11]
Target: 1 - testing sources [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Target: 5 - testing sources [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]


Target: 2 - testing sources [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Target: 6 - testing sources [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11]

---------------------------- (1) include target candidates


---------------------------- (1) include target candidates
Target: 0 - testing sources [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]candidate set: [(4, 1), (4, 2)]


candidate set: [(5, 1

testing candidate: (7, 1) maximum statistic, n_perm: 200
candidate set current source: [(7, 1), (7, 2)]
testing candidate: (7, 2) maximum statistic, n_perm: 200
testing candidate: (5, 2) maximum statistic, n_perm: 200
testing candidate: (7, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(6, 1), (6, 2)]
testing candidate: (6, 2) maximum statistic, n_perm: 200
candidate set current source: [(6, 1), (6, 2)]
testing candidate: (6, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(8, 1), (8, 2)]
testing candidate: (8, 1) maximum statistic, n_perm: 200
testing candidate: (6, 2) maximum statistic, n_perm: 200
testing candidate: (7, 2) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(8, 1), (8, 2)]
testing candidate: (8, 2) maximum statistic, n_perm: 200
testing candidate: (7, 2) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(7, 1), (7, 2)]
testing can


---------------------------- (2) include source candidates
 -- significant
candidate set current source: [(0, 1), (0, 2)]

---------------------------- (4) final statisticstesting candidate: (0, 1) maximum statistic, n_perm: 200

selected variables: [(4, 1), (4, 2), (7, 1), (7, 2)]
omnibus test, n_perm: 500
 -- significant

sequential maximum statistic, n_perm: 500
testing candidate: (9, 2) maximum statistic, n_perm: 200

---------------------------- (2) include source candidates
candidate set current source: [(0, 1), (0, 2)]
testing candidate: (0, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(3, 1), (3, 2)]
testing candidate: (3, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(1, 1), (1, 2)]
testing candidate: (1, 2) maximum statistic, n_perm: 200
 -- significant

sequential maximum statistic, n_perm: 500
 -- not significant
candidate set current source: [(1, 1), (1, 2)]
testing candidate: (1, 2) maximum st

 -- not significant
candidate set current source: [(4, 1), (4, 2)]
testing candidate: (4, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(4, 1), (4, 2)]
testing candidate: (4, 2) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(10, 1), (10, 2)]
testing candidate: (10, 2) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(11, 1), (11, 2)]
testing candidate: (11, 2) maximum statistic, n_perm: 200
testing candidate: (4, 2) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(5, 1), (5, 2)]
testing candidate: (5, 1) maximum statistic, n_perm: 200
 -- significant

---------------------------- (4) final statistics
selected variables: [(7, 1), (7, 2), (6, 1), (10, 2), (10, 1)]
omnibus test, n_perm: 500
 -- not significant
candidate set current source: [(11, 1), (11, 2)]
testing candidate: (11, 2) maximum statistic, n_perm: 200
 -- not significant

-------------

final target samples: [(3, 1), (3, 2)]



Stopping sequential max stats at candidate with rank 0.
final source samples: [(3, 1), (5, 1), (7, 1), (11, 2)]
final target samples: [(8, 1), (8, 2)]


final source samples: [(0, 1), (0, 2), (6, 1), (9, 1)]
final target samples: [(10, 1), (10, 2)]



Stopping sequential max stats at candidate with rank 0.

Stopping sequential max stats at candidate with rank 0.
final source samples: [(0, 1), (0, 2), (3, 1), (4, 1), (4, 2), (5, 1), (5, 2)]
final target samples: [(11, 1), (11, 2)]



Target: 0 - testing sources [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
---------------------------- (1) include target candidates

Target: 1 - testing sources [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

Target: 2 - testing sources [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11]
candidate set: [(0, 1), (0, 2)]

---------------------------- (1) include target candidates

Target: 3 - testing sources [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11]

Target: 4 - testing sources [0, 1, 2, 3, 5, 6, 7, 8, 9, 10,

...

testing candidate: (2, 2) minimum statistic, n_perm: 500
 -- significant

sequential maximum statistic, n_perm: 500
 -- significant
selected vars sources [(1, 1), (2, 1), (2, 0), (4, 1), (4, 0), (6, 1), (6, 0), (8, 1), (8, 0), (9, 1), (9, 0), (10, 1), (10, 0), (11, 1), (11, 0)]
selected candidates current source: [(4, 1), (4, 2)]
testing candidate: (4, 2) minimum statistic, n_perm: 500
 -- significant
selected vars sources [(1, 1), (2, 1), (2, 0), (4, 1), (4, 0), (6, 1), (6, 0), (8, 1), (8, 0), (9, 1), (9, 0), (10, 1), (10, 0), (11, 1), (11, 0)]
selected candidates current source: [(6, 1), (6, 2)]
testing candidate: (6, 2) minimum statistic, n_perm: 500
 -- significant
selected vars sources [(1, 1), (2, 1), (2, 0), (4, 1), (4, 0), (6, 1), (6, 0), (8, 1), (8, 0), (9, 1), (9, 0), (10, 1), (10, 0), (11, 1), (11, 0)]
selected candidates current source: [(8, 1), (8, 2)]
testing candidate: (8, 2) minimum statistic, n_perm: 500
 -- significant
selected vars sources [(1, 1), (2, 1), (2, 0), (4

selected candidates current source: [(7, 1), (7, 2)]
testing candidate: (7, 2) minimum statistic, n_perm: 500
 -- not significant

---------------------------- (3) prune candidates
selected vars sources [(1, 0), (1, 1), (2, 1), (2, 0), (3, 1), (3, 0), (4, 1), (4, 0), (6, 0), (6, 1), (7, 1), (7, 0)]
selected candidates current source: [(1, 2), (1, 1)]
testing candidate: (1, 1) minimum statistic, n_perm: 500
candidate set current source: [(2, 1), (2, 2)]
testing candidate: (2, 1) maximum statistic, n_perm: 200
testing candidate: (2, 2) maximum statistic, n_perm: 200
candidate set current source: [(3, 1), (3, 2)]
testing candidate: (3, 1) maximum statistic, n_perm: 200
testing candidate: (3, 2) maximum statistic, n_perm: 200
 -- significant
selected vars sources [(1, 0), (1, 1), (2, 1), (2, 0), (3, 1), (3, 0), (4, 1), (4, 0), (6, 0), (6, 1), (7, 1), (7, 0)]
selected candidates current source: [(2, 1), (2, 2)]
testing candidate: (2, 2) minimum statistic, n_perm: 500
 -- significant
selecte

selected candidates current source: [(2, 1), (2, 2)]
testing candidate: (2, 2) minimum statistic, n_perm: 500
 -- significant
selected vars sources [(1, 0), (2, 1), (2, 0), (4, 1), (4, 0), (5, 0), (5, 1), (9, 0), (9, 1)]
selected candidates current source: [(4, 1), (4, 2)]
testing candidate: (4, 2) minimum statistic, n_perm: 500
 -- significant
selected vars sources [(1, 0), (2, 1), (2, 0), (4, 1), (4, 0), (5, 0), (5, 1), (9, 0), (9, 1)]
selected candidates current source: [(5, 2), (5, 1)]
testing candidate: (5, 1) minimum statistic, n_perm: 500
 -- significant
selected vars sources [(1, 0), (2, 1), (2, 0), (4, 1), (4, 0), (5, 0), (5, 1), (9, 0), (9, 1)]
selected candidates current source: [(9, 2), (9, 1)]
testing candidate: (9, 1) minimum statistic, n_perm: 500
 -- significant

---------------------------- (4) final statistics
selected variables: [(11, 1), (11, 2), (1, 2), (2, 1), (2, 2), (4, 1), (4, 2), (5, 2), (5, 1), (9, 2), (9, 1)]
omnibus test, n_perm: 500
 -- significant

sequen


---------------------------- (2) include source candidates
candidate set: [(0, 1), (0, 2), (1, 1), (1, 2), (2, 1), (2, 2), (3, 1), (3, 2), (4, 1), (4, 2), (5, 1), (5, 2), (6, 1), (6, 2), (8, 1), (8, 2), (9, 1), (9, 2), (10, 1), (10, 2), (11, 1), (11, 2)]
testing candidate: (4, 1) maximum statistic, n_perm: 200
testing candidate: (7, 2) maximum statistic, n_perm: 200
testing candidate: (2, 1) maximum statistic, n_perm: 200
testing candidate: (0, 2) maximum statistic, n_perm: 200
testing candidate: (7, 1) maximum statistic, n_perm: 200
testing candidate: (5, 2) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (3) prune source candidate
selected candidates: [(1, 1), (9, 2), (11, 2)]
testing candidate: (11, 2) minimum statistic, n_perm: 500
testing candidate: (4, 2) maximum statistic, n_perm: 200
testing candidate: (2, 2) maximum statistic, n_perm: 200
 -- significant

---------------------------- (4) final statistics
selected variables: [(6, 1), (6, 2), (1

testing candidate: (9, 2) maximum statistic, n_perm: 200
testing candidate: (2, 1) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (3) prune source candidate
selected candidates: [(4, 1), (4, 2), (3, 1), (1, 2), (3, 2)]
 -- not significant

---------------------------- (3) prune source candidate
selected candidates: [(2, 1), (2, 2), (9, 1), (9, 2), (4, 1), (4, 2)]
testing candidate: (3, 2) minimum statistic, n_perm: 500
testing candidate: (4, 2) minimum statistic, n_perm: 500
testing candidate: (1, 1) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (3) prune source candidate
selected candidates: [(4, 1), (4, 2), (6, 1), (1, 1), (6, 2), (5, 1), (3, 1), (5, 2)]
testing candidate: (5, 2) minimum statistic, n_perm: 500
testing candidate: (0, 2) maximum statistic, n_perm: 200
 -- significant

---------------------------- (4) final statistics
selected variables: [(9, 1), (9, 2), (4, 1), (4, 2), (3, 1), (1, 2), (3, 2)]
omnibus 